----유저----
모든 유저는 구매자 또는 판매자가 될 수 있음.
모든 유저의 행동(구매, 판매)는 로그로 남겨놓음.
1. 유저등록
2. 모든 유저 조회(customer_id, seller_id 같이 조회할 수 있게) 

----아이템 검색----
추후 다양한 필터(제조사, 제품군별 세부스펙, 가격 범위 등)를 적용하여 소비자 편의를 향상시킬 수 있음.

가격 오름차순, 내림차순 조회 가능하도록
1. 아이템 등록
2. 아이템 조회(All, by Category_id, by seller_id)

----주문----
1. 배송정보, 지불방식(결제방식, 할인 적용 판단), 주문상품리스트 입력받아 주문한다.
2. 주문상품리스트 바탕으로 order_item테이블에 element생성
3. 주문취소(여유 있으면 추가)

4. 주문조회(전체, by customer_id)

ERD 링크 : https://www.erdcloud.com/d/xpuTkFPnALWXkzgLY

In [ ]:
import sqlite3 as sl
import pandas as pd
con = sl.connect('db ver2.db')
cur = con.cursor()

In [ ]:
# pandas로 결과보기
def showResultByPandas(res):
  rows = res

  cols = [column[0] for column in cur.description]

  data_df = pd.DataFrame.from_records(data=rows, columns=cols)

  return data_df

--------------회원 관련-------------------

In [ ]:
import sqlite3
from datetime import datetime
now = datetime.now()

def create_user(name, address, telnum, create_by = 'admin', change_by='admin'):  #회원 등록 
  date = str(now.date())
  my_conn.execute("INSERT INTO `user` (`user_name`, `user_address`, `user_telnum`, `enrolld_date`, `create_on`, `change_on`) VALUES ( '"
  + name +"', '"+ address+"', '" + telnum +"', '"+ date + "', '" + date +"', '" + date +"' )"  
  )

print("Do you want to create User? (answer yes or no): ")
ans = input()
if ans =='yes':
  print("Put username, address, telephone number:")
  user_name, user_add, user_tel= input().split()
  create_user(user_name, user_add, user_tel)

In [ ]:
# 권호님이 만든 회원추가메서드 수정 했습니다.

from datetime import datetime
now = datetime.now()

def create_user(name, address, telnum):  #회원 등록 
  date = str(now.date())
  cur.execute("INSERT INTO `user` (`name`, `address`, `telnum`, `enroll_date`) VALUES (?, ?, ?, ?)", (name, address, telnum, now)
  )


print("Do you want to create User? (answer yes or no): ")
ans = input()
if ans =='yes':
  print("Put username, address, telephone number:")
  user_name, user_add, user_tel= input().split(',')
  create_user(user_name, user_add, user_tel)

In [ ]:
def view_all_user():         #모든 유저 조회 /w custoer_id, seller_id
  r_set = my_conn.execute("""
  SELECT A.id, A.user_name, B.seller_id, C.customer_id FROM user A 
  LEFT JOIN (SELECT id as seller_id, user_id FROM seller) B
  ON A.id = B.user_id
  LEFT JOIN (SELECT id as customer_id, user_id FROM customer) C 
  ON A.id = C.user_id;
  """)
  for row in r_set:
     print(row) 
view_all_user()

In [ ]:
# customer 등록 using user_id
def create_customer(user_id):
  cur.execute('INSERT INTO customer (user_id) VALUES (?)',(user_id,))

In [ ]:
# seller 등록 using user_id
def create_seller(user_id):
  cur.execute('INSERT INTO seller (user_id) VALUES (?)',(user_id,))

--------------회원 관련-------------------

--------------주문 관련-------------------

In [ ]:
from datetime import datetime
from sqlalchemy.exc import SQLAlchemyError
now = str(datetime.now())


def make_order(customerid, paymentid,city_name,street_name,zipcode_number): 

  q="SELECT DISTINCT(id)  FROM delivery"
  r_set=my_conn.execute(q)
  data=r_set.fetchall()
  delivery_id=str(len(data)+1)
  is_delete=str(0)
  my_conn.execute("INSERT INTO `order` (`customer_id`, `delivery_id`, `payment_id`, `create_on`, `change_on`,`is_delete`) VALUES ( '"
  + customerid +"', '"+ delivery_id+"', '" + paymentid +"', '" + now +"', '" + now +"', '" + is_delete +"' )"
)
  q="SELECT DISTINCT(id)  FROM 'order'"
  r_set=my_conn.execute(q)
  data=r_set.fetchall()
  order_id=str(len(data))

  status="before delivery"
  my_conn.execute("INSERT INTO `delivery` (`status`, `city`, `street`, `zipcode`,`create_on`, `change_on`) VALUES ( '"
  + status +"', '"+ city_name+"', '" + street_name +"', '" + zipcode_number +"', '" + now +"', '" + now +"' )"
)
  print("Enter the number of products you want.")
  number=int(input())
  for i in range(number):
    print("Enter the ID of the product you want.")
    item_id=str(input())
    my_conn.execute("INSERT INTO `orderitem` (`order_id`, `item_id`,`create_on`, `change_on`,`is_delete`) VALUES ( '"
    + order_id +"', '"+ item_id+"','" + now +"', '" + now +"', '" + is_delete +"' )"
  )

print("If you want to make order,put your customer id, payment method(1: credit Card, 2: KaKao Pay, 3: Paypal, 4: Account Transfer)and address information (city, street, zipcode).")
customer_id, payment_id,city,street,zipcode= input().split()
make_order(customer_id, payment_id,city,street,zipcode)

In [ ]:
#makeOrder
# item_list :: [[item0_id, item0_count], [item1_id, item1_count], ... ]
def makeOrder(customer_id, item_list, payment_method, discount, city, street, zipcode):
  # create payment data
  q = 'insert into payment (payment,discount) values (?,?)'
  cur.execute(q,(payment_method, discount))
  payment_id = cur.lastrowid
  
  # create delivery data
  q = 'insert into delivery (status, city, street, zipcode) values (?,?,?,?)'
  cur.execute(q, ('preparing', city, street, zipcode))
  delivery_id = cur.lastrowid

  # create order
  q = 'insert into orders (customer_id, delivery_id, payment_id, is_delete) values (?,?,?,?)'
  cur.execute(q, (customer_id, delivery_id, payment_id, 0))
  order_id = cur.lastrowid

  # create order item           // item_list :: [[item0_id, item0_count], [item1_id, item1_count], ... ]
  for item in item_list:
    item_id = item[0]
    item_count = item[1]
    
    # item_id로 item테이블에서 데이터 찾기 (가격, 재고량)
    cur.execute('select distinct price, stock from item where id = ?', (item_id,))
    res = cur.fetchone()
    item_price = res[0]
    item_stock = res[1]

    # order_item주문할 상품 저장.    price(결제금액) = 상품가격 - 할인
    q = 'insert into order_item (order_id, item_id, count, price) values (?,?,?,?)'
    cur.execute(q, (order_id, item_id, item_count, item_price*item_count - discount))

    # 주문 후 재고량 변경 쿼리
    q = 'UPDATE item SET stock = ? where id = ?'
    cur.execute(q,(item_stock - item_count, item_id))
  

In [ ]:
def findAllOrders(con):
  #주문번호, 고객번호, 상품 이름, 상품 갯수, 총 가격, 배송 정보  모두 조회 후 반환
  cur.execute('''SELECT o.id , o.customer_id, i.name, oi.count, oi.price, d.status  
  FROM orders o, delivery d, payment p, order_item oi, item i 
  where d.id = o.delivery_id and p.id = o.payment_id and oi.order_id = o.id and oi.item_id = i.id
  ''')
  res = cur.fetchall()
  return res

In [ ]:
def findOrderByCustomerId(customer_id):
  # 특정 고객(customer_id)이 주문한 상품 모두 조회
  cur.execute('''SELECT o.id , o.customer_id, i.name, oi.count, oi.price, d.status  
  FROM orders o, delivery d, payment p, order_item oi, item i 
  where d.id = o.delivery_id and p.id = o.payment_id and oi.order_id = o.id and oi.item_id = i.id and o.customer_id = :id
  ''', id = customer_id)
  res = cur.fetchall()
  return res

--------------주문 관련-------------------

--------------상품 관련-------------------

In [ ]:
# add item method
def addItem(con, category_id, item_name, manufacturing,item_price, item_stock, seller_id):
  q = 'insert into item (category_id, name, price, stock, seller_id) values (?,?,?,?,?)'
  cur.execute(q,(category_id, item_name, item_price, item_stock, seller_id))
  item_id = cur.lastrowid
  if category_id == 1:
    q = 'insert into cpu (manufacturing, item_id) value s (?,?)'
    cur.execute(q,(manufacturing, item_id))
  elif category_id ==2: 
    q = 'insert into gpu (manufacturing, item_id) values (?,?)'
    cur.execute(q,(manufacturing, item_id))
  elif category_id ==3:
    q = 'insert into memory (manufacturing, item_id) values (?,?)'
    cur.execute(q,(manufacturing, item_id))
  elif category_id ==4:
    q = 'insert into power (manufacturing, item_id) values (?,?)'
    cur.execute(q,(manufacturing, item_id)) 

In [ ]:
# 모든 아이템 조회
def findAllItem():
  res = cur.execute('select * from item').fetchall()
  return res

In [ ]:
# 특정 카테고리의 상품 검색 오름차순 정렬
def findItemByItemCategory(category_id):
  if category_id == 1:
    q = 'select i.name, c.manufacturing, i.price, i.stock from item i, cpu c where i.id = c.item_id order by i.price asc'
    res = cur.execute(q).fetchall()
    return res
  elif category_id ==2:
    q = 'select i.name, g.manufacturing, i.price, i.stock from item i, gpu g where i.id = g.item_id order by i.price asc'
    res = cur.execute(q).fetchall()
    return res
  elif category_id ==3:
    q = 'select i.name, p.manufacturing, i.price, i.stock from item i, power p where i.id = p.item_id order by i.price asc'
    res = cur.execute(q).fetchall()
    return res
  elif category_id ==4:
    q = 'select i.name, m.manufacturing, i.price, i.stock from item i, memory m where i.id = m.item_id order by i.price asc'
    res = cur.execute(q).fetchall()
    return res

--------------상품 관련-------------------

In [ ]:
#sample data for component tables
from sqlalchemy import create_engine
my_conn = create_engine("sqlite:////content/db ver3.db")


r_set = my_conn.execute('''INSERT INTO cpu (item_id, model_name, manufacturing, is_delete, clock ) VALUES
(1, 'RYZEN 5600', 'AMD', 0,3),
(2, 'RYZEN 5600X', 'AMD',0, 3),
(3, 'RYZEN 5800X', 'AMD',0, 4),
(4, 'RYZEN 5900X', 'AMD',0, 2),
(5, 'RYZEN 7600X', 'AMD',0, 3),
(6, 'RYZEN 5950X', 'AMD',0, 5),
(7, 'RYZEN 3800X', 'AMD',0, 5),
(8, 'RYZEN 3200G', 'AMD',0, 6),
(9, 'RYZEN 3700X', 'AMD',0, 7),

(10, 'i7-12700F', 'INTEL', 0,3),
(11, 'i9-12900K', 'INTEL', 0,4),
(12, 'i9-12900KS', 'INTEL',0, 4),
(13, 'i9-10900X', 'INTEL',0, 5),
(14, 'i9-9900K', 'INTEL', 0,6);
''')

r_set = my_conn.execute('''INSERT INTO gpu (item_id, model_name, chipset, manufacturing,  is_delete ) VALUES
(1, 'RTX 4090 O24G GAMING OC D6X 24GB', 'RTX 4090', 'ASUS', 0),
(2, 'RTX 4080 GAMEROCK OC D6X 16GB', 'RTX 4080', 'EMTEK', 0),
(3, 'RTX 3060 Gaming OC V2 D6 12GB', 'RTX 3060', 'GIGABYTE', 0),
(4, 'ASUS TUF Gaming GEFORCE RTX 3070 Ti O8G OC D6X 8GB', 'RTX 3070 Ti', 'ASUS', 0),
(5, 'GALAX GEFORCE RTX 3060 Ti D6 8GB LHR', 'RTX 3060 Ti', 'GALAX',0),
(6, 'ASUS TUF Gaming GEFORCE GTX 1660 SUPER 6G D6 6GB', 'GTX 1660 SUPER', 'ASUS',  0),
(7, 'GALAX GEFORCE GTX 1660 Ti BLACK D D6 6GB', 'GTX 1660 Ti', 'GALAX',  0),
(8, 'GIGABYTE GEFORCE GTX 1660 UDV OC D5 6GB', 'GTX 1660', 'GIGABYTE', 0), 
(9, 'ASUS DUAL GEFORCE RTX 2070 SUPER D6 8GB', 'RTX 2070 SUPER', 'ASUS',  0),
(10, 'ASUS DUAL GEFORCE RTX 2070 D6 8GB', 'RTX 2070', 'ASUS', 0),
(11, 'GIGABYTE GEFORCE GTX 1650 UD2 OC D5 4GB Nano LP', 'GTX 1650', 'GIGABYTE', 0);
''')
r_set = my_conn.execute('''INSERT INTO memory (item_id, model_name, capacity, manufacturing, is_delete) VALUES
(1, 'DDR4-3200 8GB', 8, 'SAMSUNG', 0),
(2, 'DDR4-3200 16GB', 16, 'SAMSUNG', 0),
(3, 'DDR4-3200 32GB', 32, 'SAMSUNG', 0),
(4, 'DDR4-3200 4GB', 4, 'SAMSUNG', 0),
(5, 'DDR5-4800 16GB', 16, 'SAMSUNG', 0),
(6, 'DDR4-4800 8GB', 8, 'SAMSUNG', 0),
(7, 'DDR4-4800 32GB', 32, 'SAMSUNG', 0),
(8, 'Crucial DDR4-3200 CL22 32GB', 32, 'MICRON', 0),
(9, 'Crucial DDR4-3200 CL22 16GB', 16, 'MICRON', 0),
(10, 'Crucial DDR4-3200 CL22 8GB', 8, 'MICRON', 0),
(11, 'Crucial DDR5-4800 CL40 32GB', 32, 'MICRON', 0),
(12, 'Crucial DDR5-4800 CL40 16GB', 16, 'MICRON', 0),
(13, 'Crucial DDR5-4800 CL40 8GB', 8, 'MICRON', 0);
''')
r_set = my_conn.execute('''INSERT INTO power (item_id, model_name, wattage, manufacturing, is_delete) VALUES
(1, 'MICRONIX Classic II FULL CHANGE 700W 80PLUS BRONZE 230V EU', 700, 'MICRONIX', 0),
(2, 'Egpu 750 GQ 80PLUS GOLD', 750, 'Egpu',  0),
(3, 'MICRONIX Classic II FULL CHANGE 600W 80PLUS BRONZE 230V EU', 600, 'MICRONIX', 0),
(4, 'MICRONIX Classic II FULL CHANGE 800W 80PLUS BRONZE 230V EU', 800, 'MICRONIX', 0),
(5, 'MICRONIX Classic II FULL CHANGE 500W 80PLUS BRONZE 230V EU', 800, 'MICRONIX',  0),
(6, 'MICRONIX Classic II 850W 80PLUS GOLD 230V EU', 850, 'MICRONIX', 0),
(7, 'MICRONIX Classic II 1050W 80PLUS GOLD 230V EU', 850, 'MICRONIX', 0),
(8, 'AcBEL M88-900', 900, 'AcBEL', 0);
''')

In [ ]:
#addcomponent() add data to component table(gpu, memory, power, cpu)
#you cand 

def addcpu(item_id, model_name, manufacturing, clock, is_delete):

  my_tuple=(item_id, model_name, manufacturing, clock, is_delete)
  q = 'INSERT INTO cpu (item_id, model_name, manufacturing, clock, is_delete) values (?,?,?,?,?)'
  r_set = my_conn.execute( q, my_tuple)


def addpower(item_id, model_name, wattage, manufacturing, is_delete):

  my_tuple= (item_id, model_name, wattage, manufacturing, is_delete)
  q = 'INSERT INTO power (item_id, model_name, wattage, manufacturing, is_delete) values (?,?,?,?,?)'
  r_set = my_conn.execute( q, my_tuple)


def addgpu(item_id, model_name, chipset, manufacturing, is_delete):

  my_tuple= (item_id, model_name, chipset, manufacturing, is_delete)
  q = 'INSERT INTO gpu (item_id, model_name, chipset, manufacturing, is_delete) values (?,?,?,?,?)'
  r_set = my_conn.execute( q, my_tuple)





def addmemory(item_id, model_name, capacity, manufacturing, is_delete):

  my_tuple= (item_id, model_name, capacity, manufacturing, is_delete)
  q = 'INSERT INTO memory (item_id, model_name, capacity, manufacturing, is_delete) values (?,?,?,?,?)'
  r_set = my_conn.execute( q, my_tuple)

def addcomponent():
  while 1:
   x=int(input('select which to add \n1. gpu 2. memory 3. cpu 4. power 5.break \n'))
   if x == 1:
     item_id = int(input('item_id'))
     model_name = input('model name\n')
     manufacturing = input('manufacturing\n')
     is_delete = int(input('is_delete\n'))
     chipset = input('chipset\n')
     addgpu(item_id, model_name, chipset, manufacturing, is_delete)
     print('added')
     
   elif x == 2:
     item_id = int(input('item_id'))
     model_name = input('model name\n')
     manufacturing = input('manufacturing\n')
     is_delete = int(input('is_delete\n'))
     capacity = int(input('capacity\n'))
     addmemory(item_id, model_name, capacity, manufacturing, is_delete)
     print('added')

   elif x == 3:
     item_id = int(input('item_id'))
     model_name = input('model name\n')
     manufacturing = input('manufacturing\n')
     is_delete = int(input('is_delete\n'))
     clock = int(input('clock'))
     addcpu(item_id, model_name, manufacturing, clock, is_delete)    
     print('added')
   elif x == 4:
     item_id = int(input('item_id'))
     model_name = input('model name\n')
     manufacturing = input('manufacturing\n')
     is_delete = int(input('is_delete\n')) 
     wattage = int(input('wattage\n'))
     addpower(item_id, model_name, wattage, manufacturing, is_delete)
     print('added')

   elif x == 5:
     break
   else :
     print('retry')
     continue

addcomponent()
24

In [ ]:
# findcomponent() finds component information from component tables(gpu, cpu, power, memory)
#
def findcomponent():
  x= int(input('1.cpu 2.power 3. memory 4. gpu\n'))
  if x==1:
    y = int(input('1.clock 2. manufacturing\n'))
    if y==1:
      cl = int(input('insert wanted lowest clock\n'))
      q = 'select item_id, model_name, manufacturing, clock, is_delete from cpu where clock >= (?)'
      r_set = my_conn.execute(q,cl)
      for i in r_set:
        print(i)
    elif y==2:
      q = 'select item_id, model_name, manufacturing, clock, is_delete from cpu where manufacturing = (?)'
      ma = input('insert wanted -manufacturer')      
      r_set = my_conn.execute(q,ma)
      for i in r_set:
        print(i)
  elif x==2:
    y = int(input('1.wattage 2. manufacturing\n'))
    if y==1:
      cl = int(input('insert wanted wattage\n'))
      q = 'select item_id, model_name, manufacturing, wattage, is_delete from power where wattage == (?)'
      r_set = my_conn.execute(q,cl)
      for i in r_set:
        print(i)
    elif y==2:
      q = 'select item_id, model_name, manufacturing, wattage, is_delete from power where manufacturing = (?)'
      ma = input('insert wanted manufacturer\n')      
      r_set = my_conn.execute(q,ma)
      for i in r_set:
        print(i)
  elif x==3:
    y = int(input('1.capacity 2. manufacturing\n'))
    if y==1:
      cl = int(input('insert wanted lowest capacity\n'))
      q = 'select item_id, model_name, manufacturing, capacity, is_delete from memory where capacity >= (?)'
      r_set = my_conn.execute(q,cl)
      for i in r_set:
        print(i)
    elif y==2:
      q = 'select item_id, model_name, manufacturing, capacity, is_delete from memory where manufacturing = (?)'
      ma = input('insert wanted manufacturer\n')      
      r_set = my_conn.execute(q,ma)
      for i in r_set:
        print(i)
  elif x==4:
    y = int(input('1.chipset 2. manufacturing\n'))
    if y==1:
      cl = input('insert wanted chipset\n')
      q = 'select item_id, model_name, manufacturing, chipset, is_delete from gpu where chipset == (?)'
      r_set = my_conn.execute(q,cl)
      for i in r_set:
        print(i)
    elif y==2:
      q = 'select item_id, model_name, manufacturing, chipset, is_delete from gpu where manufacturing = (?)'
      ma = input('insert wanted manufacturer\n')      
      r_set = my_conn.execute(q,ma)
      for i in r_set:
        print(i)
  else :
    print('error')
        
findcomponent()

In [ ]:
#activity_log 관련 메서드

from sqlalchemy import create_engine
my_conn = create_engine('sqlite:////content/db.db')

r_set=my_conn.execute('''
CREATE TABLE `activity_log` (
	`id`	integer	NOT NULL primary key autoincrement,
	`user_id`	integer	NOT NULL,
	`log_date`	text	NULL,
	`activity`	text	NULL,
	`result`	text	NULL,
	`ip_address`	text	NULL,
	`create_on`	text	NULL,
	`create_by`	integer	NULL,
	`change_on`	text	NULL,
	`change_by`	integer	NULL,
  foreign key (user_id) references user (id)
);
''')

my_conn.execute('''
INSERT INTO 'activity_log' ('id', 'user_id', 'log_date', 'activity', 'result', 'ip_address') VALUES 
('1', '1', '2021-06-24 05:20:52', 'order', 'complete', '192.168.0.1'),
('2', '15', '2021-06-28 05:20:52', 'order', 'complete', '192.168.0.2'),
('3', '13', '2021-06-30 05:20:52', 'order', 'complete', '192.168.0.5'),
('4', '4', '2021-07-01 05:20:52', 'order', 'complete', '192.168.0.8'),
('5', '17', '2021-07-05 05:20:52.', 'order', 'complete', '192.168.0.11'),
('6', '1', '2021-07-12 05:20:52', 'sell', 'not complete', '192.168.0.1'),
('7', '5', '2021-07-14 05:20:52.', 'order', 'not complete', '192.168.0.4'),
('8', '9', '2021-07-17 05:20:52.', 'sell', 'complete', '192.168.0.13'),
('9', '2', '2021-07-25 05:20:52', 'sell', 'complete', '192.168.0.17'),
('10', '8', '2021-07-26 05:20:52', 'sell', 'not complete', '192.168.0.7');
''')

In [ ]:
r_set=my_conn.execute("SELECT * from activity_log")

for row in r_set:
  print(row)

In [ ]:
r_set=my_conn.execute("SELECT user_id, activity, result from activity_log WHERE activity = 'order'")

for row in r_set:
  print(row)

In [ ]:
r_set=my_conn.execute("SELECT user_id, activity, result from activity_log WHERE activity = 'sell'")

for row in r_set:
  print(row)

In [ ]:
r_set=my_conn.execute("SELECT user_id, activity, result from activity_log WHERE result = 'complete'")

for row in r_set:
  print(row)

In [ ]:
r_set=my_conn.execute("SELECT user_id, activity, result from activity_log WHERE result = 'not complete'")

for row in r_set:
  print(row)